# DOFBOT Fruit Sorting – YOLOv8 + 6 Classes
This notebook loads the DOFBOT arm + YOLO model and sorts 6 fruits:
apple, banana, orange, pear, strawberry, watermelon.


In [ ]:
import time
from Arm_Lib import Arm_Device
from ultralytics import YOLO

arm = Arm_Device()
time.sleep(0.2)

model = YOLO("best.pt")
print("Arm connected.")
print("Model classes:", model.names)

In [ ]:
def open_gripper():
    arm.Arm_serial_servo_write(6,60,400); time.sleep(0.4)

def close_gripper():
    arm.Arm_serial_servo_write(6,135,400); time.sleep(0.4)

def move_arm(angles,duration=800):
    for i in range(5):
        sid=i+1; val=angles[i]
        t=int(duration*1.2) if sid==5 else duration
        arm.Arm_serial_servo_write(sid,val,t); time.sleep(0.01)
    time.sleep(duration/1000)

def go_up():
    arm.Arm_serial_servo_write(2,90,1000)
    arm.Arm_serial_servo_write(3,90,1000)
    arm.Arm_serial_servo_write(4,90,1000)
    time.sleep(0.8)

def go_home():
    move_arm(home_pos,900)

print("Helpers ready.")

In [ ]:
home_pos=[90,130,0,0,90]
pick_above=[90,80,50,50,270]
pick_pos=[90,53,33,36,270]

basket_apple=[136,66,20,29,270]
basket_banana=[65,22,64,56,270]
basket_orange=[117,19,66,56,270]
basket_pear=[44,66,20,28,270]
basket_strawberry=[150,60,22,30,270]
basket_watermelon=[30,60,22,30,270]

print("Positions loaded.")

In [ ]:
fruit_to_basket={
    "apple":basket_apple,
    "banana":basket_banana,
    "orange":basket_orange,
    "pear":basket_pear,
    "strawberry":basket_strawberry,
    "watermelon":basket_watermelon,
}

def basket_for_class(name):
    key=name.strip().lower()
    if key in fruit_to_basket: return fruit_to_basket[key]
    print("[WARN] No basket for",name); return None

print("Mapping ready.")

In [ ]:
def pick_and_place(target,label=""):
    if target is None: return
    print("[INFO] Sorting:",label)

    open_gripper(); go_home()
    move_arm(pick_above,1000)
    move_arm(pick_pos,1000)
    close_gripper()
    move_arm(pick_above,1000)
    move_arm(target,1000)
    open_gripper()
    go_up(); go_home()
    print("[INFO] Done.\n")

In [ ]:
def detect_fruit_class():
    results=model.predict(source=0,imgsz=480,conf=0.6,verbose=False)
    if not results or len(results[0].boxes)==0:
        print("[INFO] No fruit detected."); return None
    boxes=results[0].boxes
    best=int(boxes.conf.argmax())
    cls=int(boxes.cls[best])
    name=str(model.names[cls]).lower()
    print("[INFO] Detected:",name)
    return name

print("Detection ready.")

In [ ]:
try:
    print("[INFO] Starting loop...")
    while True:
        fruit=detect_fruit_class()
        if fruit is None: continue
        target=basket_for_class(fruit)
        if target is None: continue
        pick_and_place(target,fruit)
except KeyboardInterrupt:
    print("[INFO] Stopped.")


In [ ]:
try:
    del arm; print("Arm deleted.")
except: print("Cleanup skipped.")